In [ ]:
from skimage import io

from skimage.filters import roberts, scharr, prewitt, sobel, hessian, frangi, sato, threshold_yen
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.exposure import equalize_hist
from skimage.measure import label, regionprops, regionprops_table
from skimage.morphology import erosion, dilation
from skimage.feature import greycomatrix, greycoprops

from matplotlib import pyplot as plt

import os
import numpy as np
import pandas as pd

from skimage.filters import gaussian


from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

import os
import glob
import math

import statistics
from scipy.stats import mannwhitneyu, f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd



In [ ]:
def lines(path, store, threshold=0.02):
    length = len(os.listdir(path))
    i=0 
    for image_path in os.listdir(path):
        if not image_path.startswith('.'):
            i = i+1
            print(f"{i} of {length-1}") #path: {path}, type: {store}")
            input_path = os.path.join(path, image_path)
            img = io.imread(input_path, as_gray=True)
            img = resize(img, (512, 256))
            imagewt = sato(img, mode='constant')
            number = np.sum(imagewt>threshold)
            name = image_path
            store[name] = number
            
            
def pvalues(a, b, c):
    F, p = f_oneway(a, b, c)
    print('Statistics=%.3f, p=%.3f' % (F, p))
    scores = a + b + c
    group = []
    for i in range(len(a)):
        group.append('WT')
    for i in range(len(b)):
        group.append('Het')
    for i in range(len(c)):
        group.append('Homo')
    df = pd.DataFrame({'score': scores, 'group': group})
    tukey = pairwise_tukeyhsd(endog=df['score'],
                             groups=df['group'],
                             alpha=0.05)
    print(tukey)  

def plotbox(a, xa, ya, ylabel, alpha=0.5, s=3.5):
    sns.set(style="white")
    boxplot_width = 0.1 # thinner to make room for having swarmplot beside
    swarmplot_offset = -0.5 # offset to left of boxplot
    xlim_offset = -1 # necessary to show leftmost swarmplot  
    colours = ['lightskyblue','coral', 'dimgrey']
    #fig = plt.figure(figsize=(6,4))

    ax = sns.swarmplot(x=xa, y=ya, data=a, palette=colours, alpha=alpha, s=s)

    path_collections = [child for child in ax.get_children() 
                        if isinstance(child,matplotlib.collections.PathCollection)] 

    for path_collection in path_collections: 
        x,y = np.array(path_collection.get_offsets()).T 
        xnew = x + swarmplot_offset
        offsets = list(zip(xnew,y)) 
        path_collection.set_offsets(offsets)

    sns.boxplot(x=xa, y=ya, data=a, width=boxplot_width, ax=ax, palette=colours) 
    def change_width(ax, new_value):
        for patch in ax.patches:
            current_width = patch.get_width()
            diff = current_width - new_value
            # change patch width
            patch.set_width(new_value)
            # re-center patch
            patch.set_x(patch.get_x() + diff * .5)
    change_width(ax,.25)
    plt.xticks([0, 1.1, 2.3], ['Wild Type', 'Heterozygous', 'Homozygous or \ncompound het'])
    ax.set_xticklabels(ax.get_xticklabels(), ha="right") # align labels to left
    ax.set_xlim(xlim_offset,ax.get_xlim()[1]) # to show leftmost swarmplot
    ax.set_xlabel('Filaggrin gene mutation status', fontsize=14)
    ax.set_ylabel(ylabel, fontsize=14)
    ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
    #plt.xticks([1, 2, 3], ['WT', '+/-', '-/-'])

###COMPARISON 0 AND 1
def addstats(result01, result12, result02, ymax):
    x1, x2 = 0, 1   # columns 'Sat' and 'Sun' (first column: 0, see plt.xticks())
    y = a[ya].max() + (a[ya].max() * 0.05 )
    h = 2
    col = 'k'

    plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
    plt.text((x1+x2)*.5, y+h, result01, ha='center', va='bottom', color=col)
    ###COMPARISON 1 AND 2
    x1, x2 = 1, 2   # columns 'Sat' and 'Sun' (first column: 0, see plt.xticks())


    y = a[ya].max() + (a[ya].max() * 0.1 )
    plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
    plt.text((x1+x2)*.5, y+h, result12, ha='center', va='bottom', color=col)

    ###COMPARISON 0 AND 1
    x1, x2 = 0, 2   # columns 'Sat' and 'Sun' (first column: 0, see plt.xticks())


    y = a[ya].max() + (a[ya].max() * 0.18 )

    plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
    plt.text((x1+x2)*.5, y+h, result02, ha='center', va='bottom', color=col)
    
def countimages(path):
    i=0
    for fname in os.listdir(path):
        if not fname.startswith('.'):
            i=i+1
            #print(fname)
    print(f"Image count for {path}: {i}")
    return i



In [ ]:
"""
PATHS
"""

base_dir = '/Users/L/Downloads/palmarhyperlinearity/'
base_dir_thenar = '/Users/L/Downloads/palmarhyperlinearity/thenarversionall2/'
base_dir_p = '/Users/L/Downloads/palmarhyperlinearity/palmarversionall2/'

path_thenar_wt = os.path.join(base_dir_thenar, 'wtupdated')
path_thenar_het = os.path.join(base_dir_thenar, 'het')
path_thenar_homo = os.path.join(base_dir_thenar, 'homo')

path_palm_wt = os.path.join(base_dir_p, 'wt')
path_palm_het = os.path.join(base_dir_p, 'het')
path_palm_homo = os.path.join(base_dir_p, 'homo')

countimages(path_thenar_wt) 
 
countimages(path_thenar_het) 
 
countimages(path_thenar_homo) 
 

countimages(path_palm_wt)
countimages(path_palm_het)
countimages(path_palm_homo)

print(f" total image count = {countimages(path_palm_wt) + countimages(path_palm_het) + countimages(path_palm_homo)}")


In [ ]:
"""
T0.02 - score generation
"""
thres = 0.02

thenar_wt_scores= {}
thenar_het_scores= {}
thenar_homo_scores= {}

lines(path_thenar_wt, thenar_wt_scores, thres)
lines(path_thenar_het, thenar_het_scores, thres) 
lines(path_thenar_homo, thenar_homo_scores, thres)

"""
P0.02 - score generation
"""
thres=0.02

palmwtscores= {} 
palmhetscores= {}
palmhomoscores= {}

lines(path_palm_wt, palmwtscores, thres)
lines(path_palm_het, palmhetscores, thres)
lines(path_palm_homo, palmhomoscores, thres)

print("done palm 0.02")



In [ ]:
"""
T0.02 - dataframe
"""

thenarwtdf = pd.DataFrame.from_dict(thenar_wt_scores, orient="index")
thenarwtdf['ID'] = thenarwtdf.index
thenarwtdf = thenarwtdf.assign(FLGstatus = 0)

thenarhetdf = pd.DataFrame.from_dict(thenar_het_scores, orient="index")
thenarhetdf['ID'] = thenarhetdf.index
thenarhetdf = thenarhetdf.assign(FLGstatus = 1)

thenarhomodf = pd.DataFrame.from_dict(thenar_homo_scores, orient="index")
thenarhomodf['ID'] = thenarhomodf.index
thenarhomodf = thenarhomodf.assign(FLGstatus = 2)




thenarscoresalldf = pd.concat([thenarwtdf, thenarhetdf, thenarhomodf])
thenarscoresalldf.set_index(['ID'])
thenarscoresalldf.columns = ['Thenar scores', 'ID', 'FLG status']

thenarscoresalldf.head()

thenarscoresalldf.to_csv("thenarscoresalldf.csv")

list_thenar_wt_scores = list(thenar_wt_scores.values())
list_thenar_het_scores  = list(thenar_het_scores.values())
list_thenar_homo_scores  = list(thenar_homo_scores.values())

pvalues(list_thenar_wt_scores , list_thenar_het_scores, list_thenar_homo_scores)

In [ ]:
"""
T0.02 - figure
"""

a = pd.read_csv("thenarscoresalldf.csv")
ylab = 'Thenar hyperlinearity score'

ya="Thenar scores"
result01 = "***"
result12 = "***"
result02 = "***"

plotbox(a=a, xa="FLG status", ya=ya, ylabel=ylab, alpha=0.5, s=3.5)
ymax = a[ya].max()
addstats(result01, result12, result02, ymax)
#plt.title('Thenar dataset')

#####add in scores from other images
x1, x2 = -1, 3   # columns 'Sat' and 'Sun' (first column: 0, see plt.xticks())
h = 2
col = 'k'

y = 12214
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=0.2, c=col)
plt.text((x1+x2)*.5, y+h, result01, ha='center', va='bottom', color=col)

y = 11855
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=0.2, c=col)
plt.text((x1+x2)*.5, y+h, result01, ha='center', va='bottom', color=col)

y = 20479
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=0.2, c=col)
plt.text((x1+x2)*.5, y+h, result01, ha='center', va='bottom', color=col)


sns.despine()
plt.show()





In [ ]:
st_dev = statistics.pstdev(list_thenar_wt_scores)
print(statistics.mean(list_thenar_wt_scores))
print("Standard deviation of the thenar wt 0.02: " + str(st_dev))
print(" ")

print(statistics.mean(list_thenar_het_scores))
st_dev = statistics.pstdev(list_thenar_het_scores)
print("Standard deviation of the thenar -/+ 0.02: " + str(st_dev))
print(" ")

print(statistics.mean(list_thenar_homo_scores))
st_dev = statistics.pstdev(list_thenar_homo_scores)
print("Standard deviation of the thenar -/- 0.02: " + str(st_dev))

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
"""
P0.02 - dataframe generation
"""
palmwtdf = pd.DataFrame.from_dict(palmwtscores, orient="index")
palmwtdf['ID'] = palmwtdf.index
palmwtdf = palmwtdf.assign(FLGstatus = 0)

palmhetdf = pd.DataFrame.from_dict(palmhetscores, orient="index")
palmhetdf['ID'] = palmhetdf.index
palmhetdf = palmhetdf.assign(FLGstatus = 1)

palmhomodf = pd.DataFrame.from_dict(palmhomoscores, orient="index")
palmhomodf['ID'] = palmhomodf.index
palmhomodf = palmhomodf.assign(FLGstatus = 2)




palmalldf = pd.concat([palmwtdf, palmhetdf, palmhomodf])
palmalldf.set_index(['ID'])

palmalldf.columns = ['Palmar scores', 'ID', 'FLG status']

palmalldf.to_csv("palmalldf.csv")
#palmalldf.head()

list_palmwtscores = list(palmwtscores.values())
list_palmhetscores = list(palmhetscores.values())
list_palmhomoscores = list(palmhomoscores.values())
pvalues(list_palmwtscores, list_palmhetscores, list_palmhomoscores)



In [ ]:
"""
P0.02 - figure
"""

a = pd.read_csv("palmalldf.csv")
ylab = 'Palmar hyperlinearity score'

ya="Palmar scores"
result01 = "***"
result12 = "***"
result02 = "***"
 
plotbox(a=a, xa="FLG status", ya=ya, ylabel=ylab, s=3, alpha=0.6)
ymax = a[ya].max()
addstats(result01, result12, result02, ymax)


######add scores
y = 17584
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=0.2, c=col)
plt.text((x1+x2)*.5, y+h, result01, ha='center', va='bottom', color=col)

y = 18636
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=0.2, c=col)
plt.text((x1+x2)*.5, y+h, result01, ha='center', va='bottom', color=col)

y = 33610
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=0.2, c=col)
plt.text((x1+x2)*.5, y+h, result01, ha='center', va='bottom', color=col)


#plt.title('Palmar dataset')
sns.despine()
plt.show()



In [ ]:
print(statistics.mean(list_palmwtscores))
st_dev = statistics.pstdev(list_palmwtscores)
print("Standard deviation of the palm wt 0.02: " + str(st_dev))
print(" ")


print(statistics.mean(list_palmhetscores))
st_dev = statistics.pstdev(list_palmhetscores)
print("Standard deviation of the palm -/+ 0.02: " + str(st_dev))
print(" ")

print(statistics.mean(list_palmhomoscores))
st_dev = statistics.pstdev(list_palmhomoscores)
print("Standard deviation of the palm -/- 0.02: " + str(st_dev))



In [ ]:
#

In [1]:
#

In [ ]:
"""
END
"""
 

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#